### 약간의 테스트
파라메터가 너무 헷갈려서, 그걸 한번 테스트해보려함 

In [1]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

In [4]:
llm = ChatOllama(
    model="gemma2:9b",
    temperature=0,
    base_url="http://host.docker.internal:11434"
    # other params...
)

In [6]:
llm.invoke("What's 7 🍎 8?")

AIMessage(content="This is a bit of a trick question!  \n\nYou can't do standard math with apples.  🍎 + 🍎 = 🍎🍎, but not a number like 15. \n\n\nLet me know if you want to try a different kind of problem! 😊 \n", additional_kwargs={}, response_metadata={'model': 'gemma2:9b', 'created_at': '2024-10-10T13:11:32.772208Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 7660926584, 'load_duration': 61644834, 'prompt_eval_count': 19, 'prompt_eval_duration': 1200365000, 'eval_count': 60, 'eval_duration': 6397142000}, id='run-ee4da525-9e76-4727-a9f8-9f426f32ad00-0', usage_metadata={'input_tokens': 19, 'output_tokens': 60, 'total_tokens': 79})

In [7]:
examples = [
    {"question": "2 🍎 4", "result": "8"},
    {"question": "3 🍎 5", "result": "15"},
    {"question": "7 🍎 3", "result": "21"},
    {"question": "What did the cow say to the moon?", "result": "nothing at all"},
    {
        "question": "Write me a poem about the moon",
        "result": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

In [8]:
to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OllamaEmbeddings(model="gemma2:9b", 
                             base_url="http://host.docker.internal:11434")
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore, k=2
)

In [9]:
example_selector.select_examples({"question": "moon"})

[{'question': 'Write me a poem about the moon',
  'result': 'One for the moon, and one for me, who are we to talk about the moon?'},
 {'question': 'What did the cow say to the moon?', 'result': 'nothing at all'}]

In [10]:
example_selector.select_examples({"question": "203 🍎 302"})

[{'question': '3 🍎 5', 'result': '15'}, {'question': '7 🍎 3', 'result': '21'}]

키값이 상관이 없음. abc 로 하던말던.. 

In [8]:
example_selector.select_examples({"abc": "moon"})

[{'question': 'Write me a poem about the moon',
  'result': 'One for the moon, and one for me, who are we to talk about the moon?'},
 {'question': 'What did the cow say to the moon?', 'result': 'nothing at all'}]

In [9]:
example_selector.select_examples({"abc": "203 🍎 302"})

[{'question': '3 🍎 5', 'result': '15'}, {'question': '7 🍎 3', 'result': '21'}]

### FewShotChatMessagePromptTemplate

위에 `examples` 데이터는 `question`과 `result`로 이뤄진 데이터인데, ai 로 바인딩할 때, 잘못된 "output"으로 매핑하면 어떨까.

In [11]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{question}"), ("ai", "{output}")]
    )
)

아래처럼 오류가 난다.

In [14]:
few_shot_prompt.invoke(input="What's 2 🍎 3000?")

KeyError: 'output'

제대로 result 로 바꾸도록 한다. input_variables 는 selector 에 쿼리로 보내는 값인 거 같은데, 뭐를 가져다놔도 상관이 없음.. 이상함.

In [23]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["question"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{question}"), ("ai", "{result}")]
    )
)

In [24]:
few_shot_prompt.invoke(input="What's 2 🍎 3000?")

ChatPromptValue(messages=[HumanMessage(content='7 🍎 3', additional_kwargs={}, response_metadata={}), AIMessage(content='21', additional_kwargs={}, response_metadata={}), HumanMessage(content='3 🍎 5', additional_kwargs={}, response_metadata={}), AIMessage(content='15', additional_kwargs={}, response_metadata={})])

aaaaa 로 키값을 바꾸더라도, 벡터데이터를 조회하는데 문제가 없다. 다만 invoke 값에 object 형으로 넣게된다면 input_variables 와 맞춰야 한다. 안그러면 에러 발생함.

In [26]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["aaaaa"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{question}"), ("ai", "{result}")]
    )
)
few_shot_prompt.invoke(input={"aaaaa":"What's 2 🍎 3000?"})

ChatPromptValue(messages=[HumanMessage(content='7 🍎 3', additional_kwargs={}, response_metadata={}), AIMessage(content='21', additional_kwargs={}, response_metadata={}), HumanMessage(content='3 🍎 5', additional_kwargs={}, response_metadata={}), AIMessage(content='15', additional_kwargs={}, response_metadata={})])

In [12]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["question"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{question}"), ("ai", "{result}")]
    )
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{question}")
    ]
)

In [29]:
final_prompt.invoke(input={"question":"30 🍎 5"})

ChatPromptValue(messages=[SystemMessage(content='You are a wondrous wizard of math.', additional_kwargs={}, response_metadata={}), HumanMessage(content='3 🍎 5', additional_kwargs={}, response_metadata={}), AIMessage(content='15', additional_kwargs={}, response_metadata={}), HumanMessage(content='7 🍎 3', additional_kwargs={}, response_metadata={}), AIMessage(content='21', additional_kwargs={}, response_metadata={}), HumanMessage(content='30 🍎 5', additional_kwargs={}, response_metadata={})])

In [13]:
chain = final_prompt | llm

In [14]:
chain.invoke("What's 7 🍎 8?")

AIMessage(content='56 🍎  \n\nRemember, "🍎" means multiplication! 😊  \n', additional_kwargs={}, response_metadata={'model': 'gemma2:9b', 'created_at': '2024-10-10T13:12:36.723334Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2530174666, 'load_duration': 57357583, 'prompt_eval_count': 62, 'prompt_eval_duration': 646020000, 'eval_count': 18, 'eval_duration': 1819768000}, id='run-064e0160-7a33-4533-ad2b-1eb89419f0a9-0', usage_metadata={'input_tokens': 62, 'output_tokens': 18, 'total_tokens': 80})